In [2]:
import numpy as np
import pandas as pd
from datetime import timedelta
import glob
import os

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
file = pd.read_csv('raw/서울특별시 대규모점포 인허가 정보.csv', encoding='cp949')
file

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3000000,2019300020207500001,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),3140.0,낙원상가,20190925150602,U,2019-09-27 02:40:00.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
1,3000000,2019300020207500002,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),3140.0,낙원상가,20190925150506,I,2019-09-27 02:22:37.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
2,3000000,2003300012907500001,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,광장주식회사,20070801173946,I,2018-08-31 23:59:59.0,시장,199837.414267,451974.128468,NaN
3,3000000,2004300007607500002,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 종로 1 (종로1가),110714.0,종로1번지아케이트,20181219132959,U,2018-12-21 02:40:00.0,전문점,197993.904628,452032.958591,대규모점포
4,3000000,2004300007607500003,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,20070721102756,I,2018-08-31 23:59:59.0,대형마트,198351.818737,451921.603026,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,3180000,2008318011707500001,20080424,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 영등포구 문래로 164 영등포 SK 리더스뷰 (문래동3가),7297.0,룩스쇼핑센터,20200907173445,U,2019-12-09 00:09:00.0,전문점,191087.758954,446157.437289,NaN
923,3180000,2001318007607500032,20011203,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 영등포구 당산로 42 (문래동3가),150991.0,홈플러스(주) 영등포점,20200907105126,U,2019-12-09 00:09:00.0,대형마트,190729.506453,446227.075041,NaN
924,3210000,2013321012207500011,19820531,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 서초구 나루터로 37 (잠원동),137907.0,신사쇼핑,20200909145703,U,2019-12-08 23:01:00.0,그 밖의 대규모점포,201230.693449,445992.459343,NaN
925,3230000,2014323019807500010,20140717,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 송파구 올림픽로 300 (신천동),138240.0,롯데월드몰,20200929141449,U,2019-10-31 00:01:00.0,쇼핑센터,209074.900840,445657.809330,NaN


In [5]:
# Nan값 확인
file.isnull().sum()

개방자치단체코드      0
관리번호          0
인허가일자         0
인허가취소일자     920
영업상태코드        0
영업상태명         0
상세영업상태코드      0
상세영업상태명       8
폐업일자        796
휴업시작일자      911
휴업종료일자      911
재개업일자       893
전화번호         15
소재지면적        75
소재지우편번호     518
지번주소        122
도로명주소       176
도로명우편번호     358
사업장명          0
최종수정일자        0
데이터갱신구분       0
데이터갱신일자       0
업태구분명         4
좌표정보(X)      74
좌표정보(Y)      74
점포구분명       236
dtype: int64

In [30]:
# 필요한 컬럼만 빼서 저장하기
## 인허가일자, 상세영업상태명, 폐업일자, 지번주소, 도로명주소, 사업장명, 최종수정일자, 데이터갱신일자

data = file[['인허가일자', '상세영업상태명', '폐업일자', '지번주소', '도로명주소', '사업장명', '최종수정일자']]
data

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20190917,정상영업,NaN,서울특별시 종로구 낙원동 288번지 낙원상가,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),낙원상가,20190925150602
1,20190917,정상영업,NaN,서울특별시 종로구 낙원동 288번지 낙원상가,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),낙원상가,20190925150506
2,20030304,정상영업,NaN,서울특별시 종로구 예지동 6번지 1호,NaN,광장주식회사,20070801173946
3,20040430,정상영업,NaN,서울특별시 종로구 종로1가 1번지,서울특별시 종로구 종로 1 (종로1가),종로1번지아케이트,20181219132959
4,20040430,정상영업,NaN,서울특별시 종로구 서린동 33호,NaN,영풍문고,20070721102756
...,...,...,...,...,...,...,...
922,20080424,정상영업,NaN,서울특별시 영등포구 문래동3가 55-16 영등포 SK 리더스뷰,서울특별시 영등포구 문래로 164 영등포 SK 리더스뷰 (문래동3가),룩스쇼핑센터,20200907173445
923,20011203,정상영업,NaN,서울특별시 영등포구 문래동3가 55번지 3호,서울특별시 영등포구 당산로 42 (문래동3가),홈플러스(주) 영등포점,20200907105126
924,19820531,정상영업,NaN,NaN,서울특별시 서초구 나루터로 37 (잠원동),신사쇼핑,20200909145703
925,20140717,정상영업,NaN,서울특별시 송파구 신천동 29번지,서울특별시 송파구 올림픽로 300 (신천동),롯데월드몰,20200929141449


In [31]:
# 컬럼 타입 확인
data.dtypes

인허가일자        int64
상세영업상태명     object
폐업일자       float64
지번주소        object
도로명주소       object
사업장명        object
최종수정일자       int64
dtype: object

In [32]:
# 백화점만 뽑아오기
department = data[(data['사업장명'].str.contains('백화점')) | (data['사업장명'].str.contains('AK')) | (data['사업장명'].str.contains('신세계타임스퀘어점'))].reset_index(drop=True)

In [33]:
# Nan값 확인
department.isnull().sum()

인허가일자       0
상세영업상태명     0
폐업일자       36
지번주소        1
도로명주소      10
사업장명        0
최종수정일자      0
dtype: int64

In [34]:
department

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20000821,정상영업,NaN,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407
1,19791227,정상영업,NaN,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209
2,20060825,정상영업,NaN,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407
3,20080822,정상영업,NaN,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636
4,20100723,정상영업,NaN,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830
5,20060424,정상영업,NaN,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302
6,19790913,폐업처리,20171227.0,서울특별시 동대문구 전농동 620번지 69호,서울특별시 동대문구 왕산로 205-0 (전농동 롯데백화점(청량리점)),롯데백화점 청량리플라자,20171229084221
7,20060424,직권취소,NaN,서울특별시 동대문구 용두동 787호,NaN,건강백화점 동의보감,20141029145710
8,20010811,정상영업,NaN,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052
9,19880514,휴업처리,NaN,서울특별시 성북구 길음동 25번지 2호,NaN,신세계백화점 미아점,20070707104737


### 1. 폐업일자 NaN 처리

In [35]:
# 컬럼 타입 변환 : 폐업일자에 맞추기
department['최종수정일자'] = department['최종수정일자'].astype(float)
department.dtypes

인허가일자        int64
상세영업상태명     object
폐업일자       float64
지번주소        object
도로명주소       object
사업장명        object
최종수정일자     float64
dtype: object

In [36]:
# df 분리
## open_null_df : 영업 중이고 폐업일자가 null인 백화점
## closed_null_df : 영업 중이 아니고 폐업일자가 null인 백화점 -> 삭제(2017년 이전 폐업된 곳)
## notnull_df : 폐업일자가 null이 아닌 백화점

open_null_df = department[(department['상세영업상태명']=='정상영업')&(department['폐업일자'].isna())].reset_index(drop=True)
closed_null_df = department[(department['상세영업상태명']!='정상영업')&(department['폐업일자'].isna())].reset_index(drop=True)
notnull_df = department[department['폐업일자'].notnull()].reset_index(drop=True)
notnull_df['최종수정일자'] = notnull_df['최종수정일자'].astype('int64')
print(len(open_null_df), len(closed_null_df), len(notnull_df))

30 6 4


In [37]:
open_null_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20000821,정상영업,NaN,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,2.020051e+13
1,19791227,정상영업,NaN,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,2.017082e+13
2,20060825,정상영업,NaN,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,2.019072e+13
3,20080822,정상영업,NaN,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,2.018071e+13
4,20100723,정상영업,NaN,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,2.017072e+13
5,20060424,정상영업,NaN,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,2.014103e+13
6,20010811,정상영업,NaN,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,2.020042e+13
7,20061214,정상영업,NaN,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,2.017042e+13
8,20050826,정상영업,NaN,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,2.020052e+13
9,20070419,정상영업,NaN,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,2.015033e+13


In [42]:
# 영업 중이고 폐업일자가 null인 백화점의 폐업일자를 0.0으로 채우기
open_null_df['폐업일자'].fillna(0.0, inplace=True)
open_null_df['최종수정일자'] = open_null_df['최종수정일자'].astype('int64')

# 상세영업상태명 : 영업 중으로 변경
open_null_df['상세영업상태명'] = '영업중'
open_null_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20000821,영업중,0.0,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407
1,19791227,영업중,0.0,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209
2,20060825,영업중,0.0,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407
3,20080822,영업중,0.0,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636
4,20100723,영업중,0.0,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830
5,20060424,영업중,0.0,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302
6,20010811,영업중,0.0,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052
7,20061214,영업중,0.0,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,20170424142249
8,20050826,영업중,0.0,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,20200522092155
9,20070419,영업중,0.0,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,20150325102313


In [43]:
# 상세영업상태명 : 폐업으로 변경
notnull_df['상세영업상태명'] = '폐업'
notnull_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,19790913,폐업,20171227.0,서울특별시 동대문구 전농동 620번지 69호,서울특별시 동대문구 왕산로 205-0 (전농동 롯데백화점(청량리점)),롯데백화점 청량리플라자,20171229084221
1,19991216,폐업,20120201.0,서울특별시 강서구 등촌동 678번지 14호,NaN,그랜드백화점(주)마트강서점,20120210104731
2,19930830,폐업,20191014.0,서울특별시 구로구 구로동 573번지,서울특별시 구로구 구로중앙로 152 (구로동),AK PLAZA,20191015092530
3,20110412,폐업,20190831.0,서울특별시 강남구 청담동 4번지 1호 외 9필지,서울특별시 강남구 도산대로 442 (청담동),SSG Food Market 백화점,20190831105733


In [44]:
# df 병합
result_df = pd.concat([open_null_df, notnull_df], axis=0).reset_index(drop=True)
result_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20000821,영업중,0.0,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407
1,19791227,영업중,0.0,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209
2,20060825,영업중,0.0,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407
3,20080822,영업중,0.0,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636
4,20100723,영업중,0.0,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830
5,20060424,영업중,0.0,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302
6,20010811,영업중,0.0,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052
7,20061214,영업중,0.0,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,20170424142249
8,20050826,영업중,0.0,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,20200522092155
9,20070419,영업중,0.0,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,20150325102313


In [45]:
# 컬럼 타입 변환 : 폐업일자를 int64로 변환
result_df = result_df.astype({'폐업일자' : 'int64'})
result_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자
0,20000821,영업중,0,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407
1,19791227,영업중,0,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407
3,20080822,영업중,0,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636
4,20100723,영업중,0,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830
5,20060424,영업중,0,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302
6,20010811,영업중,0,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052
7,20061214,영업중,0,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,20170424142249
8,20050826,영업중,0,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,20200522092155
9,20070419,영업중,0,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,20150325102313


In [47]:
# 업태구분명 컬럼 추가
result_df['업태구분명'] = '백화점'
result_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자,업태구분명
0,20000821,영업중,0,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407,백화점
1,19791227,영업중,0,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209,백화점
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407,백화점
3,20080822,영업중,0,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636,백화점
4,20100723,영업중,0,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830,백화점
5,20060424,영업중,0,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302,백화점
6,20010811,영업중,0,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052,백화점
7,20061214,영업중,0,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,20170424142249,백화점
8,20050826,영업중,0,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,20200522092155,백화점
9,20070419,영업중,0,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,20150325102313,백화점


In [48]:
# 폐업일자가 2017년 이전인 row 삭제
fin_df = result_df[(result_df['폐업일자']==0)|(result_df['폐업일자']>20170101)].reset_index(drop=True)
fin_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자,업태구분명
0,20000821,영업중,0,서울특별시 중구 회현동1가 204번지 메사빌딩,서울특별시 중구 남대문시장10길 2 (회현동1가메사빌딩),신세계백화점메사점,20200506215407,백화점
1,19791227,영업중,0,NaN,서울특별시 중구 남대문로 81 (소공동),롯데백화점 본점,20170818143209,백화점
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40번지 999호,서울특별시 용산구 한강대로23길 55 (한강로3가),아이파크(I PARK)백화점,20190717161407,백화점
3,20080822,영업중,0,서울특별시 광진구 자양동 227번지 342호 롯데백화점 스타시티점,서울특별시 광진구 능동로 92 (자양동),롯데백화점 건대스타시티점,20180705084636,백화점
4,20100723,영업중,0,서울특별시 동대문구 전농1동 591번지 53호,서울특별시 동대문구 왕산로 214 (전농동),롯데백화점 청량리역사점,20170721103830,백화점
5,20060424,영업중,0,서울특별시 동대문구 용두동 787번지 동의보감타워,NaN,건강백화점 동의보감,20141029145302,백화점
6,20010811,영업중,0,서울특별시 성북구 길음동 20번지 1호,서울특별시 성북구 동소문로 315 (길음동 현대백화점미아점),현대백화점 미아점,20200422154052,백화점
7,20061214,영업중,0,서울특별시 강북구 미아동 70번지 6 호,서울특별시 강북구 도봉로 62 (미아동),롯데백화점 미아점,20170424142249,백화점
8,20050826,영업중,0,서울특별시 노원구 상계동 713호,서울특별시 노원구 동일로 1414 (상계동),롯데백화점 노원점,20200522092155,백화점
9,20070419,영업중,0,서울특별시 노원구 중계동 506번지 1 호,NaN,건영옴니백화점,20150325102313,백화점


In [49]:
# 데이터 저장
fin_df.to_csv('data/백화점(cp949).csv', encoding='cp949', index=False)

### 2. 2017~2019년, 2020년 데이터 분리

In [71]:
file = pd.read_csv('data/백화점(주소추가, cp949).csv', encoding='cp949')
file

,No,No.1,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,최종수정일자,업태구분명,경도,위도,상태,경도.1,지번주소.1,도로명주소.1
0,1,1,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,2.020050e+13,백화점,126.979591,37.560366,정좌표,NaN,NaN,NaN
1,2,2,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,2.017080e+13,백화점,126.981730,37.564730,정좌표,NaN,NaN,NaN
2,3,3,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,2.019070e+13,백화점,126.957153,37.529550,인근좌표,NaN,NaN,NaN
3,4,4,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,2.018070e+13,백화점,127.076807,37.537882,인근좌표,NaN,NaN,NaN
4,5,5,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,2.017070e+13,백화점,127.049229,37.580232,정좌표,NaN,NaN,NaN
5,6,6,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,2.014100e+13,백화점,127.037649,37.577788,정좌표,NaN,NaN,NaN
6,7,7,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,2.020040e+13,백화점,127.028698,37.608365,인근좌표,NaN,NaN,NaN
7,8,8,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,2.017040e+13,백화점,127.030400,37.614558,인근좌표,NaN,NaN,NaN
8,9,9,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,2.020050e+13,백화점,127.061051,37.655135,정좌표,NaN,NaN,NaN
9,10,10,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,2.015030e+13,백화점,127.064230,37.639927,정좌표,NaN,NaN,NaN


In [72]:
# 필요없는 컬럼 삭제
file.drop(['No', 'No.1', '최종수정일자', '상태', '경도.1', '지번주소.1', '도로명주소.1'], axis=1, inplace=True)
file

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927


#### 1) 2017년 영업여부 수정

In [91]:
df_2017 = file[(file['폐업일자']==0) | (file['폐업일자']>20170101)]
df_2017 = df_2017[df_2017['인허가일자']<20180101]
df_2017['년'] = 2017
df_2017

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366,2017
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2017
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550,2017
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882,2017
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232,2017
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788,2017
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365,2017
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558,2017
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135,2017
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927,2017


In [92]:
open_df = df_2017[df_2017['폐업일자'] >= 20180101].reset_index(drop=True)
closed_df = df_2017[(df_2017['폐업일자'] == 0) |  (df_2017['폐업일자'] < 20180101) & (df_2018['폐업일자'] > 20170101)].reset_index(drop=True)
open_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,19930830,폐업,20191014,서울특별시 구로구 구로동 573,서울특별시 구로구 구로중앙로 152,AK PLAZA,백화점,126.882815,37.501165,2017
1,20110412,폐업,20190831,서울특별시 강남구 청담동 4,서울특별시 강남구 도산대로 430,SSG Food Market 백화점,백화점,127.043305,37.523403,2017


In [93]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업중'
open_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,19930830,영업중,0,서울특별시 구로구 구로동 573,서울특별시 구로구 구로중앙로 152,AK PLAZA,백화점,126.882815,37.501165,2017
1,20110412,영업중,0,서울특별시 강남구 청담동 4,서울특별시 강남구 도산대로 430,SSG Food Market 백화점,백화점,127.043305,37.523403,2017


In [94]:
fin_2017_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2017_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2017_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
29,19730410,영업중,0,서울특별시 중구 충무로1가 52-2,서울특별시 중구 충무로1가 퇴계로 77,신세계백화점,백화점,126.981196,37.560012,2017
30,19790913,폐업,20171227,서울특별시 동대문구 전농동 620-1,서울특별시 동대문구 왕산로 194,롯데백화점 청량리플라자,백화점,127.044753,37.579828,2017
3,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2017
25,19850318,영업중,0,서울특별시 강남구 압구정동 494,서울특별시 강남구 압구정로 343,갤러리아백화점(서관),백화점,127.040030,37.528552,2017
20,19850708,영업중,0,서울특별시 강남구 압구정동 515,서울특별시 강남구 압구정로 407,갤러리아백화점(동관),백화점,127.041545,37.528071,2017
23,19851130,영업중,0,서울특별시 강남구 압구정동 429,서울특별시 강남구 압구정로 165,(주)현대백화점 본점,백화점,127.027446,37.527325,2017
24,19881010,영업중,0,서울특별시 강남구 삼성동 159,서울특별시 강남구 영동대로 513,현대백화점 무역센터점,백화점,127.058795,37.512512,2017
26,19881112,영업중,0,서울특별시 송파구 잠실동 40,서울특별시 송파구 올림픽로 212,롯데백화점 잠실점,백화점,127.093894,37.511095,2017
17,19910501,영업중,0,서울특별시 영등포구 영등포동 618-1,서울특별시 영등포구 영등포동3가 경인로 849,롯데백화점 영등포점,백화점,126.907196,37.516486,2017
13,19921119,영업중,0,서울특별시 서대문구 창천동 30-1,서울특별시 서대문구 연세로 13,현대백화점 신촌점,백화점,126.936552,37.556754,2017


#### 2) 2018년 영업 여부 수정

In [95]:
df_2018 = file[(file['폐업일자']==0) | (file['폐업일자']>20180101)]
df_2018 = df_2018[df_2018['인허가일자']<20190101]
df_2018['년'] = 2018
df_2018

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366,2018
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2018
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550,2018
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882,2018
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232,2018
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788,2018
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365,2018
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558,2018
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135,2018
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927,2018


In [96]:
open_df = df_2018[df_2018['폐업일자'] >= 20190101].reset_index(drop=True)
closed_df = df_2018[(df_2018['폐업일자'] == 0) |  (df_2018['폐업일자'] < 20190101) & (df_2018['폐업일자'] > 20180101)].reset_index(drop=True)
closed_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366,2018
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2018
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550,2018
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882,2018
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232,2018
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788,2018
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365,2018
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558,2018
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135,2018
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927,2018


In [97]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업중'
open_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,19930830,영업중,0,서울특별시 구로구 구로동 573,서울특별시 구로구 구로중앙로 152,AK PLAZA,백화점,126.882815,37.501165,2018
1,20110412,영업중,0,서울특별시 강남구 청담동 4,서울특별시 강남구 도산대로 430,SSG Food Market 백화점,백화점,127.043305,37.523403,2018


In [98]:
fin_2018_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2018_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2018_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
30,19730410,영업중,0,서울특별시 중구 충무로1가 52-2,서울특별시 중구 충무로1가 퇴계로 77,신세계백화점,백화점,126.981196,37.560012,2018
3,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2018
26,19850318,영업중,0,서울특별시 강남구 압구정동 494,서울특별시 강남구 압구정로 343,갤러리아백화점(서관),백화점,127.040030,37.528552,2018
21,19850708,영업중,0,서울특별시 강남구 압구정동 515,서울특별시 강남구 압구정로 407,갤러리아백화점(동관),백화점,127.041545,37.528071,2018
24,19851130,영업중,0,서울특별시 강남구 압구정동 429,서울특별시 강남구 압구정로 165,(주)현대백화점 본점,백화점,127.027446,37.527325,2018
25,19881010,영업중,0,서울특별시 강남구 삼성동 159,서울특별시 강남구 영동대로 513,현대백화점 무역센터점,백화점,127.058795,37.512512,2018
27,19881112,영업중,0,서울특별시 송파구 잠실동 40,서울특별시 송파구 올림픽로 212,롯데백화점 잠실점,백화점,127.093894,37.511095,2018
18,19910501,영업중,0,서울특별시 영등포구 영등포동 618-1,서울특별시 영등포구 영등포동3가 경인로 849,롯데백화점 영등포점,백화점,126.907196,37.516486,2018
13,19921119,영업중,0,서울특별시 서대문구 창천동 30-1,서울특별시 서대문구 연세로 13,현대백화점 신촌점,백화점,126.936552,37.556754,2018
0,19930830,영업중,0,서울특별시 구로구 구로동 573,서울특별시 구로구 구로중앙로 152,AK PLAZA,백화점,126.882815,37.501165,2018


#### 3) 2019년 영업 여부 수정

In [99]:
df_2019 = file[(file['폐업일자']==0) | (file['폐업일자']>20190101)]
df_2019 = df_2019[df_2019['인허가일자']<20200101]
df_2019['년'] = 2019
df_2019

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366,2019
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2019
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550,2019
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882,2019
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232,2019
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788,2019
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365,2019
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558,2019
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135,2019
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927,2019


In [100]:
open_df = df_2019[df_2019['폐업일자'] >= 20200101].reset_index(drop=True)
closed_df = df_2019[(df_2019['폐업일자'] == 0) |  (df_2019['폐업일자'] < 20200101) & (df_2018['폐업일자'] > 20190101)].reset_index(drop=True)
open_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년


In [101]:
fin_2019_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2019_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2019_df

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
28,19730410,영업중,0,서울특별시 중구 충무로1가 52-2,서울특별시 중구 충무로1가 퇴계로 77,신세계백화점,백화점,126.981196,37.560012,2019
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2019
24,19850318,영업중,0,서울특별시 강남구 압구정동 494,서울특별시 강남구 압구정로 343,갤러리아백화점(서관),백화점,127.040030,37.528552,2019
19,19850708,영업중,0,서울특별시 강남구 압구정동 515,서울특별시 강남구 압구정로 407,갤러리아백화점(동관),백화점,127.041545,37.528071,2019
22,19851130,영업중,0,서울특별시 강남구 압구정동 429,서울특별시 강남구 압구정로 165,(주)현대백화점 본점,백화점,127.027446,37.527325,2019
23,19881010,영업중,0,서울특별시 강남구 삼성동 159,서울특별시 강남구 영동대로 513,현대백화점 무역센터점,백화점,127.058795,37.512512,2019
25,19881112,영업중,0,서울특별시 송파구 잠실동 40,서울특별시 송파구 올림픽로 212,롯데백화점 잠실점,백화점,127.093894,37.511095,2019
16,19910501,영업중,0,서울특별시 영등포구 영등포동 618-1,서울특별시 영등포구 영등포동3가 경인로 849,롯데백화점 영등포점,백화점,126.907196,37.516486,2019
11,19921119,영업중,0,서울특별시 서대문구 창천동 30-1,서울특별시 서대문구 연세로 13,현대백화점 신촌점,백화점,126.936552,37.556754,2019
29,19930830,폐업,20191014,서울특별시 구로구 구로동 573,서울특별시 구로구 구로중앙로 152,AK PLAZA,백화점,126.882815,37.501165,2019


In [102]:
# 2017~2019년 데이터 병합
fin_df = pd.concat([fin_2017_df, fin_2018_df, fin_2019_df], axis=0).reset_index(drop=True)
fin_df = fin_df[['년', '상세영업상태명', '폐업일자', '사업장명', '업태구분명', '지번주소', '도로명주소', '위도', '경도']]
fin_df

,년,상세영업상태명,폐업일자,사업장명,업태구분명,지번주소,도로명주소,위도,경도
0,2017,영업중,0,신세계백화점,백화점,서울특별시 중구 충무로1가 52-2,서울특별시 중구 충무로1가 퇴계로 77,37.560012,126.981196
1,2017,폐업,20171227,롯데백화점 청량리플라자,백화점,서울특별시 동대문구 전농동 620-1,서울특별시 동대문구 왕산로 194,37.579828,127.044753
2,2017,영업중,0,롯데백화점 본점,백화점,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,37.564730,126.981730
3,2017,영업중,0,갤러리아백화점(서관),백화점,서울특별시 강남구 압구정동 494,서울특별시 강남구 압구정로 343,37.528552,127.040030
4,2017,영업중,0,갤러리아백화점(동관),백화점,서울특별시 강남구 압구정동 515,서울특별시 강남구 압구정로 407,37.528071,127.041545
...,...,...,...,...,...,...,...,...,...
88,2019,영업중,0,주식회사 현대백화점 디큐브시티,백화점,서울특별시 구로구 신도림동 692,서울특별시 구로구 경인로 662,37.508573,126.888873
89,2019,영업중,0,뉴코아백화점,백화점,서울특별시 서초구 잠원동 70,서울특별시 서초구 잠원로 37-48,37.508343,127.005955
90,2019,영업중,0,롯데백화점 에비뉴엘 월드타워점,백화점,서울특별시 송파구 신천동 29,서울특별시 송파구 올림픽로 300,37.513092,127.103409
91,2019,영업중,0,신세계백화점 강남점,백화점,서울특별시 서초구 반포동 19-1,서울특별시 서초구 신반포로 162,37.504878,127.002066


In [103]:
# 데이터 저장
fin_df.to_csv('data/FIN/백화점 2017-2019(전처리 완료, cp949).csv', encoding='cp949', index=False)

#### 4) 2020년 영업 여부 수정

In [104]:
df_2020 = file[(file['폐업일자']==0) | (file['폐업일자']>20200101)]
df_2020 = df_2020[df_2020['인허가일자']<20210101]
df_2020['년'] = 2020
df_2020

,인허가일자,상세영업상태명,폐업일자,지번주소,도로명주소,사업장명,업태구분명,경도,위도,년
0,20000821,영업중,0,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,신세계백화점메사점,백화점,126.979591,37.560366,2020
1,19791227,영업중,0,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,롯데백화점 본점,백화점,126.981730,37.564730,2020
2,20060825,영업중,0,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,아이파크(I PARK)백화점,백화점,126.957153,37.529550,2020
3,20080822,영업중,0,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,롯데백화점 건대스타시티점,백화점,127.076807,37.537882,2020
4,20100723,영업중,0,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,롯데백화점 청량리역사점,백화점,127.049229,37.580232,2020
5,20060424,영업중,0,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,건강백화점 동의보감,백화점,127.037649,37.577788,2020
6,20010811,영업중,0,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,현대백화점 미아점,백화점,127.028698,37.608365,2020
7,20061214,영업중,0,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,롯데백화점 미아점,백화점,127.030400,37.614558,2020
8,20050826,영업중,0,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,롯데백화점 노원점,백화점,127.061051,37.655135,2020
9,20070419,영업중,0,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,건영옴니백화점,백화점,127.064230,37.639927,2020


In [106]:
# 컬럼 정렬
df_2020 = df_2020[['년', '상세영업상태명', '폐업일자', '사업장명', '업태구분명', '지번주소', '도로명주소', '위도', '경도']]
df_2020

,년,상세영업상태명,폐업일자,사업장명,업태구분명,지번주소,도로명주소,위도,경도
0,2020,영업중,0,신세계백화점메사점,백화점,서울특별시 중구 회현동1가 204,서울특별시 중구 회현동1가 남대문시장10길 2,37.560366,126.979591
1,2020,영업중,0,롯데백화점 본점,백화점,서울특별시 중구 소공동 1,서울특별시 중구 남대문로 81,37.564730,126.981730
2,2020,영업중,0,아이파크(I PARK)백화점,백화점,서울특별시 용산구 한강로3가 40-1,서울특별시 용산구 한강로3가 청파로 56-12,37.529550,126.957153
3,2020,영업중,0,롯데백화점 건대스타시티점,백화점,서울특별시 광진구 자양동 227-1,서울특별시 광진구 아차산로 306,37.537882,127.076807
4,2020,영업중,0,롯데백화점 청량리역사점,백화점,서울특별시 동대문구 전농동 587,서울특별시 동대문구 왕산로 214,37.580232,127.049229
5,2020,영업중,0,건강백화점 동의보감,백화점,서울특별시 동대문구 용두동 787,서울특별시 동대문구 왕산로 128,37.577788,127.037649
6,2020,영업중,0,현대백화점 미아점,백화점,서울특별시 성북구 길음동 20-1,서울특별시 성북구 동소문로 315,37.608365,127.028698
7,2020,영업중,0,롯데백화점 미아점,백화점,서울특별시 강북구 미아동 70-2,서울특별시 강북구 도봉로 62,37.614558,127.030400
8,2020,영업중,0,롯데백화점 노원점,백화점,서울특별시 노원구 상계동 713,서울특별시 노원구 동일로 1414,37.655135,127.061051
9,2020,영업중,0,건영옴니백화점,백화점,서울특별시 노원구 중계동 506,서울특별시 노원구 동일로203가길 29,37.639927,127.064230


In [107]:
# 데이터 저장
df_2020.to_csv('data/FIN/백화점 2020(전처리 완료, cp949).csv', encoding='cp949', index=False)